In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing DataFrame


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns

In [364]:
file1 = open("/content/drive/MyDrive/source files/finefoods_training.txt", 'r', encoding="ISO-8859-1")
data=file1.read()
print(data)

product/productId: B001E4KFG0
review/userId: A3SGXH7AUHU8GW
review/profileName: delmartian
review/helpfulness: 1/1
review/score: 5.0
review/time: 1303862400
review/summary: Good Quality Dog Food
review/text: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.

product/productId: B00813GRG4
review/userId: A1D87F6ZCVE5NK
review/profileName: dll pa
review/helpfulness: 0/0
review/score: 1.0
review/time: 1346976000
review/summary: Not as Advertised
review/text: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".

product/productId: B000LQOCH0
review/userId: ABXLMWJIXXAIN
review/profileName: Natalia Corres "Natalia Corres"
review/helpfulness: 1/1


 **STEP 1 - I imported the data into colab and using "open" I opened the file and read the data.**

In [ ]:
import re
df=open("/content/drive/MyDrive/source files/finefoods_training.txt","r",encoding="ISO-8859-1")
lines = df.readlines()
df.close()

# remove /n at the end of each line
for index, line in enumerate(lines):
      lines[index] = line.strip()
df=pd.DataFrame(lines)
a=df[0].str.split(":")


col_1=[]
for i in range(0,18000,9):
  z=a[i][1]
  col_1.append(z)
col_1=pd.DataFrame(col_1)


col_2=[]
for b in np.arange(1,18000,9):
  h=a[b][1]
  col_2.append(h)
col_2=pd.DataFrame(col_2)


col_3=[]
for c in np.arange(2,18000,9):
  f=a[c][1]
  col_3.append(f)
col_3=pd.DataFrame(col_3)


col_4=[]
for d in np.arange(3,18000,9):
  s=a[d][1]
  col_4.append(s)
col_4=pd.DataFrame(col_4)


col_5=[]
for e in np.arange(4,18000,9):
  q=a[e][1]
  col_5.append(q)
col_5=pd.DataFrame(col_5)


col_6=[]
for w in np.arange(5,18000,9):
  r=a[w][1]
  col_6.append(r)
col_6=pd.DataFrame(col_6)


col_7=[]
for t in np.arange(6,18000,9):
  y=a[t][1]
  col_7.append(y)
col_7=pd.DataFrame(col_7)


col_8=[]
for u in np.arange(7,18000,9):
  o=a[u][1]
  col_8.append(o)
col_8=pd.DataFrame(col_8)

**STEP 2 - I used hard coding method here and sperated all the contents of the previous file into seprate columns**

In [ ]:
first_2=col_1.join(col_2,lsuffix='_left', rsuffix='_right')
third_and_four=col_3.join(col_4,lsuffix='_left', rsuffix='_right')
fifth_and_sixth=col_5.join(col_6,lsuffix='_left', rsuffix='_right')
seven_and_eight=col_7.join(col_8,lsuffix='_left', rsuffix='_right')
first_4=first_2.join(third_and_four,lsuffix='_left', rsuffix='_right')
next_4=fifth_and_sixth.join(seven_and_eight,lsuffix='_left', rsuffix='_right')
df=first_4.join(next_4,lsuffix='_left', rsuffix='_right')
df.columns=["product/productId","review/userId","review/profileName","review/helpfulness","review/score","review/time","review/summary","review/text"]


**STEP 3-I created the dataframe by merging all the separate dataframes**

In [365]:
df["Is the product rated good"]=pd.to_numeric(df['review/score'])>2.5
df['Is the product rated good']=df['Is the product rated good'].replace(True,1)
df['Is the product rated good']=df['Is the product rated good'].replace(False,0)

**STEP 4 -1 - I created a column which shows if the review of the product was good or bad. I kept the threshold at 2.5**

In [366]:
import datetime
df['review/NormalTime'] = pd.to_numeric(df['review/time']).apply(lambda x: datetime.datetime.utcfromtimestamp(x))

**STEP 4-2- I converted the unix time given in the dataframe into normal time**

# Text Analytics


In [367]:
import spacy
nlp=spacy.load("en_core_web_sm")

In [368]:
def count_pos1(text):
  doc = nlp(text)

    # Count occurrences of each part of speech
  pos_count = {}
  for token in doc:
    pos_count[token.pos_] = pos_count.get(token.pos_, 0) + 1

  return pos_count
df['pos_counts'] = df['review/summary'].apply(count_pos1)

**STEP 5-1 - Using Spacy I counted the total number of pos(parts of speech) in the summary of the reviews.**

In [369]:
def count_adjectives(text):
    # Process the text using spaCy
    doc = nlp(text)

    # Count adjectives in each sentence
    adjective_counts = []
    for sentence in doc.sents:
        adjectives = sum(1 for token in doc if token.pos_ == 'ADJ')
        #adjective_counts.append(len(adjectives))

    return adjectives

# Apply the function to each row in the DataFrame
df['adjective_counts'] = df['review/text'].apply(count_adjectives)

**STEP 5 - 2 - In this step I found the total number of adjectives a reviewer has used in his review.**

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import opinion_lexicon

# Download the lexicon if not already downloaded
nltk.download('opinion_lexicon')
def get_positive_adjectives(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    positive_words = set(opinion_lexicon.positive())

    # Count positive adjectives
    count = sum(1 for token in tokens if token.lower() in positive_words and nltk.pos_tag([token])[0][1] == 'JJ')

    return count

# Apply the function to each row in the DataFrame
df['PositiveAdjectives'] = df['review/text'].apply(get_positive_adjectives)


**STEP 5 - 3 - I realised nltk has a separate library which contains all the positive adjectives. I used that as the hint stated.**

In [371]:
def count_negative_adjectives(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Get negative words from the lexicon
    negative_words = set(opinion_lexicon.negative())

    # Count negative adjectives
    count = sum(1 for token in tokens if token.lower() in negative_words and nltk.pos_tag([token])[0][1] == 'JJ')

    return count

# Apply the function to each row in the DataFrame
df['NegativeAdjectiveCount'] = df['review/text'].apply(count_negative_adjectives)


**STEP 5 - 3 - Same thing used for negative library**

In [372]:
df

,product/productId,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text,Is the product rated good,pos_counts,adjective_counts,total_pos_count,total_pos_count_in_text,review/NormalTime,PositiveAdjectives,NegativeAdjectiveCount
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,5.0,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned ...,1,"{'SPACE': 1, 'PROPN': 4}",4,5,5,2011-04-27,1,0
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0/0,1.0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanu...,0,"{'SPACE': 1, 'PART': 1, 'ADP': 1, 'VERB': 1}",3,4,4,2012-09-07,0,0
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1/1,4.0,1219017600,"""Delight"" says it all",This is a confection that has been around a f...,1,"{'SPACE': 1, 'PUNCT': 2, 'PROPN': 1, 'VERB': 1...",8,7,7,2008-08-18,0,0
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3/3,2.0,1307923200,Cough Medicine,If you are looking for the secret ingredient ...,0,"{'SPACE': 1, 'PROPN': 1, 'NOUN': 1}",3,3,3,2011-06-13,1,0
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0/0,5.0,1350777600,Great taffy,Great taffy at a great price. There was a wi...,1,"{'SPACE': 1, 'ADJ': 1, 'NOUN': 1}",5,3,3,2012-10-21,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,B001E5E29A,A3IJNH61DY5N9K,Sharon Allan,10/11,5.0,1265846400,Great Waffles for us non-cooks,I have to laugh at the reviews that said it w...,1,"{'SPACE': 1, 'PROPN': 3, 'ADP': 1, 'NOUN': 3}",10,8,8,2010-02-11,0,0
1996,B001E5E29A,A35W3JQYP0M655,"Ambergris ""John Thomas""",7/7,5.0,1279929600,Excellent Fluffy Pancakes.....,I had read some favorable reviews of this pan...,1,"{'SPACE': 1, 'PROPN': 3, 'PUNCT': 1}",27,5,5,2010-07-24,6,0
1997,B001E5E29A,A3CLWZGMC7M9Z1,"Hippie2MARS ""Hippie2MARS""",6/6,4.0,1250640000,Good Basic Mix,I was expecting great things based on the rev...,1,"{'SPACE': 1, 'PROPN': 3}",6,4,4,2009-08-19,4,0
1998,B001E5E29A,A2V6P1FKRHCVQQ,S. Jackson,8/9,4.0,1265760000,"Good pancakes, lots of work",I love this pancake mix. I bought my first c...,1,"{'SPACE': 1, 'ADJ': 1, 'NOUN': 3, 'PUNCT': 1, ...",7,7,7,2010-02-10,1,1


# Model Fitting
In this step I just fit my data into Naive Bayes Model.

In [376]:
X=np.vstack((df['adjective_counts'],df['NegativeAdjectiveCount'],df['PositiveAdjectives']))
y=df['review/score']
X=X.reshape(2000,3)

**THE SELECTION**
:I selected only these three columns as they seem to be the most apt ones to give me the highest accuracy rate. The models fit prefectly to the selections as we want to find out the product ratings if they are good or bad. The adjectives left by the reviewers and are they positive or not.

In [377]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

The train and test data have been initialized in this step.

In [378]:
from sklearn.naive_bayes import MultinomialNB

# Create and train the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)
from sklearn.metrics import accuracy_score

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6525


Accuracy with Naive Bayes is 65.25%